# Data Jointure V2

## Configuration

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experiment: ExperimentConfig = configuration.experiments(0)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@1ca1ca9c
session = org.apache.spark.sql.SparkSession@1ca1ca9c
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output...

## Chargement des données

In [9]:
val flightDFPath = s"${configuration.common.output.basePath}/common/data/wban_enriched_flights.parquet"
val flightDF = spark.read.parquet(flightDFPath)
println("Flight DF Count: ", flightDF.count())

// Charger les données météo prétraitées depuis le parquet
val rawWeatherPath = s"${configuration.common.output.basePath}/common/data/raw_weather.parquet"
val weatherDF = spark.read.parquet(rawWeatherPath)
println("Weather DF Count: ", weatherDF.count())


(Flight DF Count: ,5780977)
(Weather DF Count: ,16446741)


flightDFPath = /home/jovyan/work/output/common/data/wban_enriched_flights.parquet
flightDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 16 more fields]
rawWeatherPath = /home/jovyan/work/output/common/data/raw_weather.parquet
weatherDF = [WBAN: string, Date: string ... 42 more fields]


[WBAN: string, Date: string ... 42 more fields]

## Restreindre les vols aux mois couverts par la météo

In [19]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val flightsCoveredMonths =
  flightDF
    .withColumn("month_utc", date_format(col("UTC_FL_DATE"), "yyyy-MM"))

val weatherDFWithConvertedDates = weatherDF.withColumn("Date", to_date(col("Date"), "yyyyMMdd"))
val weatherMonths =
  weatherDFWithConvertedDates
    .withColumn("month_utc", date_format(col("Date"), "yyyy-MM"))
    .select("month_utc").distinct()

val flightDF_mCovered =
  flightsCoveredMonths.join(weatherMonths, Seq("month_utc"), "left_semi")

val kept = flightDF_mCovered.count()
val total = flightDF.count()
println(f"[Step1] Flights in covered months: $kept / $total (${kept.toDouble * 100.0 / total}%.2f%%)")

[Step1] Flights in covered months: 1930962 / 5780977 (33.40%)


flightsCoveredMonths = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 17 more fields]
weatherDFWithConvertedDates = [WBAN: string, Date: date ... 42 more fields]
weatherMonths = [month_utc: string]
flightDF_mCovered = [month_utc: string, FL_DATE: date ... 17 more fields]
kept = 1930962
total = 5780977


5780977

In [15]:
flightsCoveredMonths.show(2, 1000 , true)

-RECORD 0---------------------------
 FL_DATE               | 2012-07-01 
 OP_CARRIER_AIRLINE_ID | 19805      
 OP_CARRIER_FL_NUM     | 1123       
 ORIGIN_AIRPORT_ID     | 10693      
 DEST_AIRPORT_ID       | 13303      
 CRS_DEP_TIME          | 600        
 ARR_DELAY_NEW         | 0.0        
 CANCELLED             | NULL       
 DIVERTED              | NULL       
 CRS_ELAPSED_TIME      | 130.0      
 WEATHER_DELAY         | NULL       
 NAS_DELAY             | NULL       
 ORIGIN_WBAN           | 13897      
 ORIGIN_TIMEZONE       | -6         
 DEST_WBAN             | 12839      
 DEST_TIMEZONE         | -5         
 UTC_CRS_DEP_TIME      | 1200       
 UTC_FL_DATE           | 2012-07-01 
 month_utc             | 2012-07    
-RECORD 1---------------------------
 FL_DATE               | 2012-07-01 
 OP_CARRIER_AIRLINE_ID | 19805      
 OP_CARRIER_FL_NUM     | 1124       
 ORIGIN_AIRPORT_ID     | 13303      
 DEST_AIRPORT_ID       | 12953      
 CRS_DEP_TIME          | 950        
 

In [16]:
weatherMonths.show(2, 1000 , true)

-RECORD 0---------
 month_utc | NULL 



In [17]:
weatherDF.show(2, 1000 , true)

-RECORD 0-----------------------------
 WBAN                      | 03011    
 Date                      | 20120101 
 Time                      | 0015     
 StationType               | 0        
 SkyCondition              | CLR      
 SkyConditionFlag          |          
 Visibility                | 10.0     
 VisibilityFlag            |          
 WeatherType               |          
 WeatherTypeFlag           |          
 DryBulbFarenheit          | 23.0     
 DryBulbFarenheitFlag      |          
 DryBulbCelsius            | -5.0     
 DryBulbCelsiusFlag        |          
 WetBulbFarenheit          | 15.0     
 WetBulbFarenheitFlag      |          
 WetBulbCelsius            | -9.5     
 WetBulbCelsiusFlag        |          
 DewPointFarenheit         | -9.0     
 DewPointFarenheitFlag     |          
 DewPointCelsius           | -23.0    
 DewPointCelsiusFlag       |          
 RelativeHumidity          | 24.0     
 RelativeHumidityFlag      |          
 WindSpeed               